In [3]:
import torch
import pyaudio
import wave
import os
import time
from AudioUtil import AudioUtil

print(torch.__version__)

model = torch.load("data/models/V8_model.pth")
model.eval()

CHUNK = 1050
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 44100

def record_audio(duration=2):
    frames = []
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNK)
    try:
        # Calculate the correct number of iterations to cover the duration
        num_frames = int((RATE / CHUNK) * duration)
        for _ in range(num_frames):
            data = stream.read(CHUNK)
            frames.append(data)
    finally:
        stream.stop_stream()
        stream.close()
        p.terminate()
    return frames

def save_temp_audio(frames, filename="temp_audio.wav"):
    with wave.open(filename, 'wb') as wf:
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(pyaudio.PyAudio().get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b''.join(frames))

def process_audio_file(filename):
    aud = AudioUtil.open(filename)
    os.remove(filename)
    reaud = AudioUtil.resample(aud, RATE)
    rechan = AudioUtil.rechannel(reaud, CHANNELS)
    dur_aud = AudioUtil.pad_trunc(rechan, 2000)
    sgram = AudioUtil.spectro_gram(dur_aud, n_mels=64, n_fft=1024, hop_len=None)

    prediction = model(sgram)
    if prediction > 0.5:
        print("Klopfen erkannt!")

print("Starte Klopfen-Erkennung...")
while True:
    start_time = time.time()
    frames = record_audio()
    save_temp_audio(frames)
    process_audio_file("temp_audio.wav")
    print(f"Time taken: {time.time() - start_time}")

2.3.0+cu118
Starte Klopfen-Erkennung...
<class 'torch.Tensor'>


TypeError: 'collections.OrderedDict' object is not callable